<a href="https://colab.research.google.com/github/fletchdr7/cadet_data_solution/blob/main/cadet_data_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##########################################
# 1) INSTALL & IMPORT LIBRARIES
##########################################

!pip install striprtf thefuzz

import pandas as pd
import numpy as np
import sqlite3
from thefuzz import process
from striprtf.striprtf import rtf_to_text
from io import StringIO
from pathlib import Path
import os

print("Libraries installed and imported.")

##########################################
# 2) UPLOAD & PARSE THE RTF FILE
##########################################

from google.colab import files

print("Please upload your .rtf file (e.g., W_DFAS_257.rtf).")
uploaded_rtf = files.upload()
rtf_filename = list(uploaded_rtf.keys())[0]
print(f"RTF file uploaded as: {rtf_filename}")

# Decode the raw bytes
rtf_bytes = uploaded_rtf[rtf_filename]
rtf_text = rtf_bytes.decode('utf-8', errors='ignore')

# Convert RTF to plain text
plain_text = rtf_to_text(rtf_text)
print("\nPreview of RTF plain text (first 500 characters):")
print(plain_text[:500])

##########################################
# 3) PARSE THE RTF TABLE (PIPE-DELIMITED)
##########################################

# We'll assume the first table is pipe-delimited (e.g. 'SSN|NAME|PGM CD|...')
# We'll collect lines until we hit a blank line

lines = plain_text.splitlines()
table_lines = []
found_header = False

for line in lines:
    if line.startswith("SSN|"):  # or whatever your header starts with
        found_header = True

    if found_header:
        if line.strip() == "":
            # blank line => end of table
            break
        table_lines.append(line)

# Join them into a single string for pandas
table_str = "\n".join(table_lines)

# Read into a DataFrame
df_rtf = pd.read_csv(StringIO(table_str), sep="|")

print("\nInitial df_rtf from RTF table:")
display(df_rtf.head())

# Remove any unnamed columns (extra columns from trailing separators)
unnamed_cols = [c for c in df_rtf.columns if "Unnamed" in c]
if unnamed_cols:
    df_rtf.drop(columns=unnamed_cols, inplace=True)
    print(f"Dropped unnamed columns: {unnamed_cols}")

##########################################
# 4) CONVERT NUMERIC COLUMNS IF NEEDED
##########################################

numeric_cols = [
    "SUBSIS    PAID",
    "BOOKS & FEES",
    "TRAINING PAID",
    "BONUS PAID",
    "COMM PAID",
    "TRN DAY",
    "COLLECTION",
    "NET PAY",
    "CUM SUBSIS",
    "CUM SUB DAY",
]
for col in numeric_cols:
    if col in df_rtf.columns:
        df_rtf[col] = pd.to_numeric(df_rtf[col], errors="coerce")

# If there's a date column like 'ROTC GRAD DATE', we can convert it as well:
if "ROTC GRAD DATE" in df_rtf.columns:
    df_rtf["ROTC GRAD DATE"] = pd.to_datetime(df_rtf["ROTC GRAD DATE"], format="%Y%m%d", errors="coerce")

print("\nAfter numeric/date conversion:")
df_rtf.info()

##########################################
# 5) UPLOAD & CLEAN THE CSV
##########################################

print("\nUpload your SMR_AS_FAVORITE.csv file:")
uploaded_csv = files.upload()
csv_filename = list(uploaded_csv.keys())[0]
print(f"CSV file uploaded as: {csv_filename}")

df_csv = pd.read_csv(csv_filename)
print("\nOriginal df_csv columns:")
print(df_csv.columns.tolist())

# If the CSV uses 'Name' instead of 'NAME', rename for consistency
if "Name" in df_csv.columns:
    df_csv.rename(columns={"Name": "NAME"}, inplace=True)

print("Updated df_csv columns:")
print(df_csv.columns.tolist())
display(df_csv.head())

##########################################
# 6) MAP THE RTF PGM CD -> AS YEAR
##########################################

# The RTF has a 'PGM CD' column that we map to AS years (e.g. SC01 -> AS100, etc.)
pgm_to_as_year = {
    "SC01": "AS100",
    "SC02": "AS200",
    "NS03": "AS300",
    "NS04": "AS400",
}

if "PGM CD" in df_rtf.columns:
    df_rtf["as_label_rtf"] = df_rtf["PGM CD"].map(pgm_to_as_year)
else:
    print("WARNING: 'PGM CD' not in df_rtf columns. Skipping as_label_rtf.")

##########################################
# 7) USE CSV'S ALREADY EXISTING 'AS Year'
##########################################

# In your CSV, you have a column 'AS Year' that might say AS100, AS200, etc.
# We'll rename that to 'as_label_csv' so it lines up with the RTF's concept:
if "AS Year" in df_csv.columns:
    df_csv.rename(columns={"AS Year": "as_label_csv"}, inplace=True)
else:
    print("WARNING: No 'AS Year' column in df_csv. We'll skip as_label_csv.")

##########################################
# 8) CREATE A COMBINED KEY (NAME + AS YEAR) FOR FUZZY MATCH
##########################################

def build_fuzzy_key(name, label):
    if pd.isna(name):
        name = ""
    if pd.isna(label):
        label = ""
    return f"{name.strip().upper()}-{label.strip().upper()}"

# Build fuzzy_key_rtf
if "NAME" in df_rtf.columns and "as_label_rtf" in df_rtf.columns:
    df_rtf["fuzzy_key_rtf"] = df_rtf.apply(
        lambda row: build_fuzzy_key(row["NAME"], row["as_label_rtf"]), axis=1
    )
else:
    print("Skipping fuzzy_key_rtf creation (missing columns).")

# Build fuzzy_key_csv
# We'll assume CSV also has 'NAME' and we just renamed 'AS Year' to 'as_label_csv'
if "NAME" in df_csv.columns and "as_label_csv" in df_csv.columns:
    df_csv["fuzzy_key_csv"] = df_csv.apply(
        lambda row: build_fuzzy_key(row["NAME"], row["as_label_csv"]), axis=1
    )
else:
    print("Skipping fuzzy_key_csv creation (missing columns).")

print("\nPreview RTF keys:")
display(df_rtf[["NAME", "PGM CD", "as_label_rtf", "fuzzy_key_rtf"]].head(10))

print("\nPreview CSV keys:")
if "fuzzy_key_csv" in df_csv.columns:
    display(df_csv[["NAME", "as_label_csv", "fuzzy_key_csv"]].head(10))
else:
    display(df_csv.head(10))

##########################################
# 9) FUZZY MATCH df_rtf KEYS AGAINST df_csv KEYS
##########################################

if "fuzzy_key_rtf" in df_rtf.columns and "fuzzy_key_csv" in df_csv.columns:
    possible_keys = df_csv["fuzzy_key_csv"].tolist()
else:
    print("No fuzzy_key columns in RTF or CSV. Fuzzy match cannot proceed.")
    possible_keys = []

best_matches = []
match_scores = []

for combined_key in df_rtf.get("fuzzy_key_rtf", []):
    result = process.extractOne(combined_key, possible_keys)
    if result is None:
        best_matches.append(None)
        match_scores.append(0)
    else:
        # thefuzz.process.extractOne usually returns (best_string, score)
        best_str, best_score = result
        best_matches.append(best_str)
        match_scores.append(best_score)

df_rtf["BEST_MATCH"] = best_matches
df_rtf["MATCH_SCORE"] = match_scores

print("\nFuzzy matching complete. Check a few rows:")
display(df_rtf[["NAME", "PGM CD", "as_label_rtf", "fuzzy_key_rtf", "BEST_MATCH", "MATCH_SCORE"]].head(20))

##########################################
# 10) MERGE THE TWO DATAFRAMES ON THE FUZZY KEY
##########################################

df_merged = pd.merge(
    df_rtf,
    df_csv,
    how="left",
    left_on="BEST_MATCH",    # best fuzzy match from df_rtf
    right_on="fuzzy_key_csv" # actual fuzzy key in df_csv
)

print("\nMerged DataFrame shape:", df_merged.shape)
print("Merged DataFrame columns:")
print(df_merged.columns.tolist())
display(df_merged.head(20))

##########################################
# 11) OPTIONAL: STORE IN SQLITE
##########################################

db_name = "cadet_data.db"
if Path(db_name).exists():
    os.remove(db_name)  # remove old copy if desired

conn = sqlite3.connect(db_name)
df_merged.to_sql("cadet_info", conn, if_exists="replace", index=False)

print(f"\nData written to SQLite DB '{db_name}' in table 'cadet_info'.")
conn.close()

print("\n--- DONE! ---")
